In [1]:
# useful to autoreload the module without restarting the kernel
%load_ext autoreload
%autoreload 2

In [3]:
from mppi import InputFiles as I, Calculators as C

# Tutorial for QeCalculator class

This tutorial describes the usage of the QeCalculator class, for performing a single calculation
with the QuantumESPRESSO.

In [9]:
all([True,False])

False

In [5]:
skip = True

In [6]:
if not skip : print('ciao')

In [11]:
len([1,2,3]) > 0

True

## Perform an scf computation for silicon

We init the PwInput object using an exsisting input file

In [4]:
inp = I.PwInput(file='IO_files/si_scf.in')
inp

{'control': {'verbosity': "'high'",
  'pseudo_dir': "'../pseudos'",
  'calculation': "'scf'",
  'prefix': "'si_scf'"},
 'system': {'force_symmorphic': '.true.',
  'occupations': "'fixed'",
  'ibrav': '2',
  'celldm(1)': '10.3',
  'ntyp': '1',
  'nat': '2',
  'ecutwfc': '40'},
 'electrons': {'conv_thr': '1e-08'},
 'ions': {},
 'cell': {},
 'atomic_species': {'Si': ['28.086', 'Si.pbe-mt_fhi.UPF']},
 'atomic_positions': {'type': 'crystal',
  'values': [['Si', [0.125, 0.125, 0.125]], ['Si', [-0.125, -0.125, -0.125]]]},
 'kpoints': {'type': 'automatic',
  'values': ([4.0, 4.0, 4.0], [0.0, 0.0, 0.0])},
 'cell_parameters': {},
 'file': 'IO_files/si_scf.in'}

To run a computation we need to instanciate the QeCalculator class, specifying the number of omp and
the mpi command used by the calculator. For instance

In [27]:
code = C.QeCalculator(omp = 2, mpi_run='mpirun -np 4', skip = True, verbose= True)

Initialize a QuantumESPRESSO calculator with OMP_NUM_THREADS=2 and command mpirun -np 4 pw.x


The global parameter of the calculator are written in the global options

In [28]:
code.global_options()

{'omp': 2,
 'mpi_run': 'mpirun -np 4',
 'executable': 'pw.x',
 'skip': True,
 'verbose': True}

The global options can be modified as follows

In [29]:
code.update_global_options(omp = 1)
code.global_options()

{'omp': 1,
 'mpi_run': 'mpirun -np 4',
 'executable': 'pw.x',
 'skip': True,
 'verbose': True}

The computation is executed by the run method. In this method we need to specify the PwInput object used as input, the 
name of the folder were the computation is executed and the name of the input file that this class write on disk before
the run. 

In general it is convenient to choose the prefix attribute of the input object as the name of the file, so the relation between
the input file and the prefix folder we the associated computation is written are evident. In thi way several computations
can be performed in the same run_dir without ambiguity, as long as each od that has a distinct prefix.

In this case we set

In [30]:
prefix = 'si_scf_test1'
inp.set_prefix(prefix)

In [34]:
result = code.run(input = inp, run_dir = 'QeCalculator_test',name=prefix)

Run directory QeCalculator_test
Skip the computation for input si_scf_test1


The arguments passed in the run method complement and overwrite the global options of code.
For istance

In [35]:
result = code.run(input = inp, run_dir = 'QeCalculator_test',name=prefix,skip=False)

delete log file: QeCalculator_test/si_scf_test1.log
delete xml file: QeCalculator_test/si_scf_test1.xml
delete folder: QeCalculator_test/si_scf_test1.save
Run directory QeCalculator_test
Executing command: mpirun -np 4 pw.x -inp si_scf_test1.in > si_scf_test1.log


so that the run is no skipped.

The specific parameters of the run are written in code.run_options. This dictionary is specific of the given run so that various run with the same code 
can be performed in different way.

In [36]:
code.run_options

{'omp': 1,
 'mpi_run': 'mpirun -np 4',
 'executable': 'pw.x',
 'skip': False,
 'verbose': True,
 'input': {'control': {'verbosity': "'high'",
   'pseudo_dir': "'../pseudos'",
   'calculation': "'scf'",
   'prefix': "'si_scf_test1'"},
  'system': {'force_symmorphic': '.true.',
   'occupations': "'fixed'",
   'ibrav': '2',
   'celldm(1)': '10.3',
   'ntyp': '1',
   'nat': '2',
   'ecutwfc': '40'},
  'electrons': {'conv_thr': '1e-08'},
  'ions': {},
  'cell': {},
  'atomic_species': {'Si': ['28.086', 'Si.pbe-mt_fhi.UPF']},
  'atomic_positions': {'type': 'crystal',
   'values': [['Si', [0.125, 0.125, 0.125]],
    ['Si', [-0.125, -0.125, -0.125]]]},
  'kpoints': {'type': 'automatic',
   'values': ([4.0, 4.0, 4.0], [0.0, 0.0, 0.0])},
  'cell_parameters': {},
  'file': 'IO_files/si_scf.in'},
 'run_dir': 'QeCalculator_test',
 'name': 'si_scf_test1'}

If the run of the simulation does not crash the result variable the data-file-schema.xml (including its relative path) for subsequent parsing.

In [37]:
result

'QeCalculator_test/si_scf_test1.save/data-file-schema.xml'

Instead, let see what happens if the simulation fails if the simuation fails. For instance if we provide an empty input to code

In [38]:
inp2 = I.PwInput()

In [39]:
prefix = 'si_scf_test2'
inp2.set_prefix(prefix)
inp2

{'control': {'prefix': "'si_scf_test2'"},
 'system': {},
 'electrons': {},
 'ions': {},
 'cell': {},
 'atomic_species': {},
 'atomic_positions': {},
 'kpoints': {},
 'cell_parameters': {}}

In [44]:
result2 = code.run(input = inp2, run_dir = 'QeCalculator_test',name=prefix) 
result2

Run directory QeCalculator_test
Executing command: mpirun -np 4 pw.x -inp si_scf_test2.in > si_scf_test2.log


In this case the output of the run method is empty

In [45]:
if result2 is None: print('is None')

is None


## Perform a nscf computation for silicon. Usage of the source_dir option

We show how to perform a pw nscf calculation using the results of the first scf run as an input.

First of all we modify the inp object to set up a nscf calculation with a given number of bands

In [59]:
inp.set_nscf(8)
prefix = 'si_nscf_test1'
inp.set_prefix(prefix)
#inp

Then, we use the code to setting up the source_dir in the run options

In [61]:
result3 = code.run(input = inp, run_dir = 'QeCalculator_test',name=prefix,source_dir='QeCalculator_test/si_scf_test1.save',skip = True)

The folder QeCalculator_test/si_nscf_test1.save already exsists. Source folder QeCalculator_test/si_scf_test1.save not copied
Run directory QeCalculator_test
Skip the computation for input si_nscf_test1


In this case result3 contain the xml data of the nscf calculation

In [62]:
result3

'QeCalculator_test/si_nscf_test1.save/data-file-schema.xml'